## Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

TensorFlow version: 2.7.0
1 Physical GPUs, 1 Logical GPUs


In [25]:
import pathlib

# Build model

In [2]:
layers.Conv2D(filters=32,
            kernel_size=3,
            strides=1,
            padding='same',
            activation='relu')(tf.random.normal([1, 28, 28, 1])).shape

TensorShape([1, 28, 28, 32])

In [13]:
layers.LSTM(units=40, return_sequences=True)(tf.random.normal([1, 28, 280])).shape

TensorShape([1, 28, 40])

In [15]:
model = tf.keras.models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation="relu", input_shape=(201, 498)),
    layers.MaxPool1D(pool_size=4, strides=4),

    layers.Conv1D(64, kernel_size=3, activation="relu"),
    layers.MaxPool1D(pool_size=2, strides=2),

    layers.Conv1D(64, kernel_size=3, activation="relu"),
    layers.Dropout(0.5),
    
    layers.LSTM(units=40, return_sequences=True),
    
    layers.MaxPool1D(pool_size=2, strides=2),
    layers.Flatten(),
    
    layers.Dense(2)
    ])
# L1 loss
model.compile(optimizer="adam",
            loss="mae",
            metrics=["mae"]
            )

In [18]:
model(tf.random.normal([1, 201, 498]))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.7392897 , -0.33533776]], dtype=float32)>

In [33]:
save_path = "model_not_trained"
# tf.saved_model.save(model, save_path)

run_model = tf.function(lambda x: model(x))
BATCH_SIZE = 1
FREQS = 201
TIME_STEPS = 498
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, FREQS, TIME_STEPS], model.inputs[0].dtype))
model.save(save_path, save_format="tf", signatures=concrete_func)

INFO:tensorflow:Assets written to: model_not_trained/assets


INFO:tensorflow:Assets written to: model_not_trained/assets


# Tinify model

In [34]:
converter = tf.lite.TFLiteConverter.from_saved_model(save_path)

save_path_tflite = os.path.basename(save_path) + ".tflite"

converter.optimizations = [tf.lite.Optimize.DEFAULT]


tflite_model = converter.convert()
tflite_model_file = pathlib.Path(save_path_tflite)
tflite_model_file.write_bytes(tflite_model)

Estimated count of arithmetic ops: 20.144 M  ops, equivalently 10.072 M  MACs


97056